# Exploratory Data Analysis

### On `Mobile money` transactions for `Mara Bank`

## Sections in this notebook.

- Introduction
    - Project Overview
    - Objectives
    - Dataset Background

- Data Importation
    - Import modules
    - Import Datasets
    - Set Global Constants

- Data Description
    - Data Information
    - Dataset Shape and Size
    - Data Types
    - Basic Statistical Summery

- Data Wrangling
    - Missing Values Analysis
    - Outlier Detection
    - Handling Inconsistencies

- Exploratory Analysis
    - Summery Statistics
    - Distribution of Data
    - Spread of Data
    - Trend Analysis
    - Correlation of Features

- Insights and Findings
    - Key Patterns
    - Anomalies
    - Business Insights
    - Recommendations


## Introduction

- Project Overview
- Objectives
- Dataset Background

### Project Overview

Imagine you’re watching millions of transactions flow through `Mara Bank` in Nigeria, like airtime purchases, money transfers, bill payments, all happening in real-time from bustling cities to quiet rural towns.

Hidden in this river of activity are patterns and sometimes, suspicious ripples that could be signs of fraud.

This project is our detective work. We’re diving into `Mara Bank’s` mobile money transaction data to uncover how people really use the `mobile banking` system and prepare the ground for building a `fraud detection model`.

### Project Objective

The Key objectives are:

- Clean the noise: Make sure the transaction data is accurate, consistent, and ready for investigation.

- Spot the patterns: Study trends in location, time, amount, and type of transactions.

### Background of Dataset

The dataset was generated mimicing the different scenerios transactions can occur in Nigeria. It contains transactions of diferrent banks, however we will be focusing on the transactions that belongs to `Mara Bank` for this project.

This dataset contains the following:

- `amount`: The value of the transaction.
- `balance`: The account balance after the transaction.
- `time`: The timestamp of the transaction.
- `holder`: The account number of the transaction's initiator or recipient.
- `kyc`: The kyc level of the account
- `holder_bvn`: The BVN of the transaction's initiator or recipient.
- `holder_bank`: The bank of the related party.
- `related`: The account number or entity related to the transaction (e.g., recipient account, ATM bank).
- `related_bvn`: The BVN of the related party.
- `related_bank`: The bank of the related party.
- `state`, `latitude`, `longitude`: Location details of the transaction.
- `status`: The outcome of the transaction (e.g., 'SUCCESS', 'FAILED').
- `type`: The transaction type (e.g., 'DEBIT', 'CREDIT').
- `category`: The specific class of transaction (e.g., 'OPENING', 'WITHDRAWAL', 'PAYMENT', 'TRANSFER', 'REVERSAL', 'BILL').
- `channel`: The channel used for the transaction (e.g., 'CARD', 'APP', 'USSD').
- `device`: The device used for the transaction (e.g., 'ATM-001', 'MOBILE-003') .
- `reference`: A unique identifier for related transactions.
- `reported`: Marks reported transactions?

## Data Importation

- Loading Required Libraries
- Reading Data Files
- Initial Data Preview

### Loading required libraries

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Import modules
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
# Load our custom library
from src.lib.analytics import analyst, engineer
from src.lib.store.load_data import load_data

### Reading data files

In [ ]:
# Load the transactions dataset for the project 
df = load_data('transactions.csv', parse_dates=['time'])

# load the accounts dataset for the project
accounts_df = load_data('accounts.csv')

### Initial data preview

In [ ]:
# Preview the dataset
df.head()

We are only interested in the data for `Steven Salazar Bank`, so we will be extracting the data.

In [ ]:
bank_name = 'Steven Salazar Bank'

In [ ]:
accounts_df.rename(columns={'latitude': 'holder_latitude', 'longitude': 'holder_longitude'}, inplace=True)
accounts_df

In [ ]:
bank_accounts = accounts_df[accounts_df['bank_name'] == bank_name]

In [ ]:
bank_df = df[df['holder_bank'] == bank_name]

In [ ]:
# Let's merge add the kyc and merchant from the account df
bank_df = bank_df.merge(bank_accounts[['kyc', 'merchant', 'account_no', 'holder_latitude', 'holder_longitude', 'opening_device']], how='left', left_on='holder', right_on='account_no').drop(columns='account_no')

In [ ]:
bank_df['holder_bvn'] = bank_df.apply(lambda row: accounts_df[(accounts_df['account_no'] == row['holder']) & (accounts_df['bank_name'] == row['holder_bank'])].squeeze()['bvn'], axis=1)

In [ ]:
def get_related_bvn(row):
    account = accounts_df[(accounts_df['account_no'] == row['related']) & (accounts_df['bank_name'] == row['related_bank'])].squeeze()
    return account['bvn'] if not account.empty else row['related_bank']

In [ ]:
bank_df['related_bvn'] = bank_df.apply(get_related_bvn, axis=1)

## Data Description

- Missing Values Analysis
- Outlier Detection
- Handling Inconsistencies

### Data Information

Let's have see the data types and shape of our dataset.

In [ ]:
# Preview the basic information about the dataset
bank_df.info()

We have let's of categorical values, this will really help with our classification.

### Dataset shape and size

In [ ]:
# Preview the size of the dataset
bank_df.shape

### Dataset Statistics

Lets summarize the dataset using summary statistics.

In [ ]:
# Set the format for pandas to display floats
pd.set_option('display.float_format', '{:.2f}'.format)

# Set the theme for our visualization
plt.style.use('dark_background')
sns.set_theme(style="darkgrid")

In [ ]:
# Sneak peak statistical summery of the dataset
bank_df.describe()

Looking at this summary, here are the things that stand out.

`Amount` 
- Mean is around 503K while median is around 93K, it skews to the right.
- Max is 9.7M which is around 15x of the 75%.
- Std is around 940K while mean is 503K, further confirming the right skew.

`Balance`
- Mean is 21M while median is 2.8M, same skew as amount.
- Max is 543B, has much gap on the 75% of more than 50x.
- Min is 54.7, no account ever got drained though some came close.

`Time`
- Ranges from Aug 2023 to Aug 2024. 
- So the dataset is for 1 year.
- We will be extracting (Day, Hour, Week, Month...) from it.

`Geo`
- Latitude ranges from 4.3 to 14.21
- Longitude ranges from -1 to 8.8
- That's Nigeria's bounding box (Lagos to Maiduguri)
- Mean lat/lon (9.5/3.7), which is central Nigeria.

`Kyc`
- The min level is 1 and max is 3
- This shows that Kyc probabily ranges from 1 to 3

`Holder Geo`
- Latitude ranges from 5.0 to 137
- Longitude ranges from -1 to 8.8
- That's Nigeria's bounding box (Lagos to Maiduguri)
- Mean lat/lon (9.7/3.6), which is central Nigeria.


## Data Wrangling

- Missing Values Analysis
- Outlier Analysis
- Handling Inconsistencies

This is a synthetic data, however just for the sake of pete let's still check if anything is missing.

In [ ]:
bank_df.isna().sum()

Not to say the obvious, there are no no missing data.

### Handling Inconsistencies.

Since we are using a synthetically generated dataset for this project, we will also be looking into the followining inconsistencies.

- Negative amount and balances.
- Since we are using reference instead of id, we will check for duplicate reference in unrelated transactions.
- Inconsistent reversals
- Is the time linear?

#### Check for negative amount and balance

In [ ]:
# First lets name our dataset
df_regular = bank_df.copy()

# Checking for negative transactions with negative amount or balance
df_regular[(df_regular['amount'] < 0) | (df_regular['balance'] < 0)]

#### Check for duplicate reference

The reference, this is used to identifier related transactions.

Transactions are related if and only if:
- A money gets transfered from one account to another (Debit - Credit)
- Money is reversed after a credit or debit.

So, given these set of rules for a relationship to exist, nonces can relate only the following number of transactions.
- 1 transaction: Single transaction eg. Opening an account, Deposits like Loan from bank
- 2 transactions: Credit - Debit, Credit - Reversal, Debit Reversal
- 4 transactions: Credit - Debit - Credit Reversal - Debit Reversal

In [ ]:
# Checking for the uniqueness of the reference
referrence_frequency = df_regular['reference'].value_counts()
referrence_frequency.value_counts()

We do not have any reference that connects unrelated transactions.

Let's check for inconsistent reverserals, to do this we need to answer this question - `Are there any reversals with a unique reference?`

In [ ]:
# Reversals with unique reference
df_regular[(df_regular['type'] == 'REVERSAL') & (df_regular['reference'].isin(referrence_frequency[referrence_frequency == 1].index))]

We don't have any irregular reversals.

#### Check if time is linear

Now is the time of the transactions linear for each account?


In [ ]:
# Check if time is linear
df_regular.groupby('holder')['time'].apply(lambda x: x.is_monotonic_increasing).value_counts()

Yes, all accounts have linear time. Good 🙂

Let's align our data types

In [ ]:
df_regular.dtypes

`Latitude`, `Longitude` are floats but should be treated as discrete variables. Hence should be converted to object.

`kyc`, `reference` are int but should be treated as categories.

In [ ]:
# Convert latitude and longitude to str
df_regular['latitude'] = df_regular['latitude'].astype(str)
df_regular['longitude'] = df_regular['longitude'].astype(str)
df_regular['holder_latitude'] = df_regular['holder_latitude'].astype(str)
df_regular['holder_longitude'] = df_regular['holder_longitude'].astype(str)


df_regular['kyc'] = df_regular['kyc'].astype(str)
df_regular['reference'] = df_regular['reference'].astype(str)

### Outlier Detection

We need our model to learn very well from the data that we will feeding it, hence we need to remove situations that way too `out of this world` from our dataset.

Let's see the spread of the dataset once again

In [ ]:
# What the shape of the dataset?
df_regular.describe()

We have removed latitude and longitude from the summary statitics, it is now treated as a discrete feature.

Let's visualize these spreads so we can understand them better.

We will be needing a special function that will help us with our visualization.

In [ ]:
# Using the boxplot to visual the spread of the numeric data
columns = df_regular.describe().columns
analyst.plotter(
    plot=lambda x:
        # Set subplots sections
        plt.subplot(int(np.ceil(len(columns))), 2, x[0] + 1)

        # Set the title of the Plot
        and plt.title(x[1])

        # Visualize the spread of the features
        and sns.boxplot(data=df_regular, x=x[1])

        # Set the label of the x-axis
        and plt.xlabel(x[1]),
    columns=columns,
    figsize=(20, 10)
)

We will handle the outliers by scaling the balanace and amount down the using log.

In [ ]:
df_regular['amount_log'] = np.log(df_regular['amount'])
df_regular['balance_log'] = np.log(df_regular['balance'])

In [ ]:
# Using the boxplot to visual the spread of the numeric data
columns = df_regular.describe().columns
analyst.plotter(
    plot=lambda x:
        # Set subplots sections
        plt.subplot(int(np.ceil(len(columns))), 2, x[0] + 1)

        # Set the title of the Plot
        and plt.title(x[1])

        # Visualize the spread of the features
        and sns.boxplot(data=df_regular, x=x[1])

        # Set the label of the x-axis
        and plt.xlabel(x[1]),
    columns=columns,
    figsize=(20, 10)
)

## Exploratory Analysis

Lets get to understand our data.

- Summary Statistics
- Correlation of Features
- Distribution of Data
- Spread of Data
- Trend Analysis 


In [ ]:
# Before we begin, let's name our df properly
df_eda = df_regular.copy()

#### Extracting Periodic features

Let's feed another curiousity we have been holding. When did most transactions occur at?

To do this we will have to extract the hour, day and month feature ealier than expected.

In [ ]:
# Extracting the hour
df_eda['hour'] = df_eda['time'].dt.hour.astype(str).str.zfill(2)

# Extracting the day
df_eda['week_day'] = df_eda['time'].dt.day_name()

# Extracting the month
df_eda['month'] = df_eda['time'].dt.month_name()

# Extracting the date
df_eda['date'] = df_eda['time'].dt.date

# Extracting the month day
df_eda['month_day'] = df_eda['time'].dt.day.astype('object')

### Summary Statistics

In [ ]:
# Let's take another look at the summary statistics
df_eda.describe()

In [ ]:
# Let's take a look at the statistics of the categorical features.
df_eda.describe(include=['object', 'bool'])

Things that stand out in this summary.

- There are `793` different accounts that transacted within the time of this transactions.

- All these `793` accounts belong to only `763` users, some users own multiple accounts.

- There are `50` related banks, which corresponds to the total number of banks in the simulation.

- There are `6` categories of tranactions that fall under the `2` types of transactions (Debit/Credit)

- A total of `7087` devices where also used accross `3` channels were used.

- There were `17028` related transactions based on the unique `reference` in the dataset.

- The transaction spans accross `37` unique states, suggesting it is all accross Nigeria.

- The most transactions took place in `Gombe`. (Ballers 💰). Is the there a hotspot there?

- The most frequent type and category of transactions are `DEBIT` and `WITHDRAWAL` respectively. Could there be more outflow than inflow?

- Most transactions were not reported, we will see the proportion later in this notebook.

- `January` is the most frequent month, though the day with most transactions is in `October` on `2025-04-06`

- `Month` day of `24` has the most frequent transactions.

- There is only 1 holder bank, `Mara Bank` so we will drop the column

- Most users are of `KYC` leve of `3`.

- `Friday` is the most active day of the week.

In [ ]:
df_eda.drop(columns='holder_bank', inplace=True)

### Correlations

Before we continue on our exploration, let's ask a vital question early.

- Do our `data` points have any form of `relationship` with each other.

- If they do, what could be the `reason` for that relationship to exist?

##### Correlation of all features

Let's see how all our features are correlating with one another.

To do this we need to encode the categorical/discrete features.

In [ ]:
df_eda.dtypes

Latitude and Longitude are numberical, however they are actually discrete variables, hence we will encode them before correlating.

Also, time is a date variable which is an ordinal and will be treated as a continious variable.

Hence our continious variables include `time`, `amount`, `balance`

##### Let's encode the discrete features so we can correlate them with other features.

We will be using a special function for this.

In [ ]:
# Get the discrete features
discrete_features = df_eda.select_dtypes(include=['object', 'bool']).columns.tolist()
discrete_features

In [ ]:
# Encode the discrete features
df_corr = engineer.encoder(df_eda, discrete_features)

# Concatinate the encoded discrete features with the non discrete features
df_corr = pd.concat([df_eda[['amount_log', 'balance_log', 'time']], df_corr], axis=1)
df_corr

In [ ]:
# Correlate the features with each other
corr_matrix = df_corr.corr(method='pearson')
corr_matrix

In [ ]:
# Plot confusion matrix 
plt.figure(figsize=(20, 10))
sns.heatmap(data=corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix')
plt.xlabel('Features')
plt.ylabel('Features')
plt.show()

Observations:

- Strong positive correlation between amount and balance suggests transaction values drive balances.

- time and date correlates well with the user features like holder, related, related_bvn, and holder_bvn hinting at potential redundancy or leakage.

- Negative correlation between month and time suggests seasonality impacts transaction timing.

- type and category show a strong positive correlation, implying categories belong to different types.

- Device usage (device) correlates moderately with time, useful for time-device fraud detection.

In [ ]:
# Plot the correlation matrix
analyst.plotter(
    plot=lambda x:
        # Set subplots sections
        plt.subplot(int(np.floor(len(corr_matrix.columns)/2)), 3, x[0] + 1)

        # Visualize the correlation of the features
        and sns.barplot(data=corr_matrix, x=corr_matrix.index, y=x[1])

        # Set the plot title
        and plt.title(f'Correlation of {x[1]}')

        # Set x lable
        and plt.xlabel('Features')

        # Set y lable
        and plt.ylabel('Correlation')
        
        and plt.tick_params(axis='x', rotation=90),

    # Set the figsize
    figsize=(20, 30),
    columns=corr_matrix.columns,
)

Visualizing the correlations, things are getting a lot more clearer.

We will now prioritize highly correlated features.

In [ ]:
interesting_correlation = analyst.bounded_correlation(corr_matrix)

In [ ]:
plt.figure(figsize=(20, 30))
analyst.plot_bounded_correlation(interesting_correlation)

Nice, now we have the correlations that we are interested in, what do we see?

- `Amount` and `Balance`: Show very strong mutual positive correlation.

- `Holder` & `Holder BVN`: Both highly correlate positively with `time` and `date`.

- `Related` & `Related BVN`: Exhibit a very strong positive correlation with each other.

- `Status` & `Type`: Are notably correlated, especially with `category`.

- `Device` & `Month`: Consistently show minimal to weak correlations overall.

Let's get the distribution of features with there correlation with other features

In [ ]:
# Get the correlations within out threshold
valued_correlations = analyst.bounded_correlation(corr_matrix)

# Count and sum the correlations of each feature
valued_correlations_count = {x: {'count': len(valued_correlations[x]), 'sum': sum(abs(valued_correlations[x]))} for x in valued_correlations}

# Convert it to a dataframe
valued_corr_df = pd.DataFrame(valued_correlations_count).transpose().sort_values(by='count', ascending=False).reset_index(names='feature')

valued_corr_df

In [ ]:
# Visualize the the correlations
plt.figure(figsize=(20, 6))
plt.title('Correlation Strength')
sns.scatterplot(data=valued_corr_df, x='count', y='sum', hue='feature')

Things that stand out from this plot:

- Strongest Correlations: Features `holder`, `time`, `holder_bvn`, `month`, and `date` show the highest correlations with `holder` as the strongest.

- Weakest Correlations: `Amount` and `category` consistently exhibit very low correlation strength across all counts.

- Variable Correlation Strength: Most features display varying correlation strengths depending on the `count` value.

- Increasing Count, Increasing Strength: For the most correlated features, strength generally increases with higher `count`.

- Outliers in Correlation: `Related` and `related_bvn` show moderate correlation, unlike others at higher counts.

### Distribution of Data

Let's visualize the distibution of the continious and discrete features

#### General View

Let's look at our data from the outside to the inside, this will enable use get a better understanding.

In [ ]:
# Let's see the details of our dataset to choose the columns to visualize
df_eda.info()

In [ ]:
# Visualize the distribution of the continious features.
columns = ['amount_log', 'balance_log', 'time']

analyst.plotter(
    plot=lambda x:
        # Set subplots sections
        plt.subplot(len(columns), 2, x[0] + 1)

        # Visualize the distribution of the features
        and sns.histplot(data=df_eda, x=x[1], kde=True)

        # Set the plot title
        and plt.title(f'Distribution of {x[1]}')

        # Set X lable
        and plt.xlabel(x[1].capitalize())

        # Set Y lable
        and plt.ylabel('Frequency'),

    # Set the figsize
    figsize=(20, 10),
    columns=columns,
)

Things that stand out.

- `Amount`: Heavily right-skewed; most transactions are for small amounts.

- `Balance`: Right-skewed, similar to amount; many transactions have low balances.

- `Time`: Frequency is relatively uniform across months (2024-2025).

- Both `amount` & `balance` distributions show a sharp drop-off after small values.

- No strong trend or seasonality evident in the time distribution plot.





In [ ]:
# Visualize the distribution of the discrete features.
columns = [
    'status',
    'type',
    'category',
    'reported',
    'channel',
    'latitude',
    'longitude',
    'hour',
    'week_day',
    'month',
    'date',
    'month_day',
    'kyc',
    'merchant'
]

analyst.plotter(
    plot=lambda x:
        # Set subplots sections
        plt.subplot(int(np.floor(len(columns))), 2, x[0] + 1)

        # Visualize the distribution of the features
        and sns.countplot(data=df_eda, x=x[1])

        # Set the plot title
        and plt.title(f'Distribution of {x[1]}')

        # Set x lable
        and plt.xlabel(x[1].capitalize())

        # Set y lable
        and plt.ylabel('frequency')

        and plt.tick_params(axis='x', rotation=90),

    # Set the figsize
    figsize=(20, 40),
    columns=columns,
)

##### Observations.

- Majority of transactions `succeed`; only a small fraction `fail`, skewing fraud detection toward anomalies.

- `Debits` heavily outnumber `credits`, but withdrawals and reversals show potential fraud-risk categories.

- Most activity comes from card-based `channels`; APP/USSD volumes are small but fraud-prone.

- `Reported` fraud cases are few, creating high class imbalance and challenge for supervised models.

- `KYC` levels are few at level 4; low levels are frequent, signaling very few people could attain the level.



Well, there we have it, there are more outflows than inflows for this through the year.

#### CashFlow

In [ ]:
# Plot cashflow
analyst.plot_cashflow(df_eda, ['hour', 'week_day', 'month', 'month_day', 'kyc'])

Observing this plot, here are the things that stand out.

- Overall negative `netflow` indicates accross all features

- `Cashflow` varies significantly by `state`, with `Kaduna` & `Ogun` showing no `netflows`.

- There is no decline of activities in `week_day`, `month` or `hour`. The is a little fluctuations in `month_day` however.

#### Distribution of features with type

In [ ]:
# Plot distribution by type
plt.figure(figsize=(20, 20))
analyst.plot_distribution(df_eda, ['channel', 'category', 'amount', 'reported', 'date', 'holder', 'kyc'], 'type')

Observations:

- There is class inbalance in `category`, it needs the `dimension` to be reduced.

- Accross all features, there is a huge gap in transaction `type`, with `debit` being almost 2x of `credit`.

#### Category Distribution

In [ ]:
# Plot distribution by category
plt.figure(figsize=(20, 20))
analyst.plot_distribution(df_eda, ['amount_log', 'balance_log', 'date', 'month_day', 'channel', 'kyc'], 'category')

Observations:

- `Amount` and `Balance`: Most transactions, especially withdrawals and deposits, are of low value.

- `Date`: Transaction frequencies show fluctuations over time, with withdrawals and deposits consistently dominant.

- Channel: APP is the primary channels, with APP heavily used for withdrawals and deposits. USSD/CARD is less utilized.

- State: Transaction volumes vary by state, with Gombe and Delter showing high activity.

- Month Day: Transaction frequencies are relatively consistent throughout the month, with no strong daily patterns for any category.

#### Channel Distribution

In [ ]:
# Plot distribution by category
plt.figure(figsize=(20, 30))
analyst.plot_distribution(df_eda, ['amount_log', 'balance_log', 'hour', 'month_day', 'channel', 'holder', 'kyc'], 'channel')

Observations: 

- Amount/Balance: Lower transaction amounts are most frequent.

- Hour: Activity levels fluctuate throughout the day for each channel.

- Month Day: Usage patterns differ across days of the month.

- Overall Channel Usage: A clear comparison of the total volume handled by each channel.

- Holder: Shows how different user relate to specific channels, with Card being the favourite option.

#### Reported Distribution

In [ ]:
plt.figure(figsize=(20, 30))
analyst.plot_distribution(df_eda, ['amount_log', 'balance_log', 'date', 'week_day', 'channel', 'category', 'holder', 'kyc'], 'reported')

- The majority of transactions involve small amounts, with frequency sharply decreasing as the transaction amount increases.

- Similarly, most account balances are relatively low, with fewer accounts holding large balances.

- "Withdrawal" transactions is significantly more frequent than other categories like "Deposit" or "Payment."

- "Opening" transactions are the least frequent among the displayed categories.

- Transactions are consistently distributed across the week, indicating a similar level of activity throughout the week.

- Mobile banking channels ("APP") appear to be the most frequently used for transactions, followed by "CARD" and then "USSD."

- Transaction frequency is relatively low for most account holders, with a few holders showing a higher frequency of transactions.

- The frequency of transactions over date shows fluctuations but no clear, strong upward or downward trend across the observed period.

- Certain states like Gombe, and Delta show a higher frequency of transactions compared to others like Abia.

### Spread of Data

In [ ]:
# Visualize the spread of the data
plt.figure(figsize=(20, 40))
analyst.plot_scatter(df_eda, 'amount_log', ['balance_log', 'channel', 'category', 'month', 'week_day', 'month_day', 'hour', 'date', 'kyc'], 'reported', 1e5)

Observations:

- No opening category of transaction was reported

- Level 3 KYC has the least reported transaction

- Transactions are not reported as much on weekends.

In [ ]:
plt.figure(figsize=(20, 40))
analyst.plot_scatter(df_eda, 'balance_log', ['amount_log', 'channel', 'category', 'month', 'week_day', 'month_day', 'hour', 'date', 'kyc'], 'reported', 1e2)

Observations:

- There is an even spread of reported transactions accross all the features.

### Trend Analysis

In [ ]:
analyst.plot_trend(df_eda, ['date', 'hour', 'month', 'week_day', 'month_day'])

Observations:

- `7am` is the most active of all the hours.
- `January` is a very active month.
- `Friday` is the most active off all days with Monday as the least. 

In [ ]:
df_eda.to_csv(f'../datasets/analyzed_transactions.csv', index=False)

In [ ]:
df_eda

## Insights and Findings
    
- Key Patterns
- Anomalies
- Business Insights
- Recommendations


### Key Patterns

- Amount ↔ Balance lockstep: Strong positive correlation; high amounts typically move balances in sync.

- Temporal uniformity: Activity is broadly even by hour/weekday/month.

- Debit & Withdrawal gravity: Debits dominate; Withdrawals are the most frequent category.

- KYC skew: KYC Level 4 heavily deconcentrated; Lower KYC levels are more frequent.

- Geo concentration: Transactions span 37 states; Gombe/Delta emerge as volume hot spots.

### Anomalies

- Heavy right-skew: Amount and balance distributions contain many high-end outliers (IQR-trimmed).

- reference integrity edge cases: Reversal/transfer pairings must be 1/2/4 events, flag any other counts.

- Cashflow dips: Net outflow pockets (e.g., KYC 3 cohorts) suggest leakage or behavioral anomalies.

- Device/Channel bursts: Localized spikes by device/channel may indicate scripted or bot-like activity.

- Time cliffs: Spikes in the morning, maybe something fishy is happening.

### Business Insights

- Risk concentration at low KYC: Disproportionate activity with low verification = elevated fraud exposure.

- Operational focus states: Gombe/Delta high volumes merit tailored limits and monitoring rules.

- Channel strategy: APP drives withdrawals; CARD steady for everyday use—tune controls per channel.

- Sparse “reported” labels: Supervised models will be label-starved; lean on anomaly/PU learning.

### Recommendations

- Tiered limits by KYC: Stricter per-txn/day caps and velocity rules for KYC 3; relax for 1,2 and 4.

- reference policy checks: Enforce 1/2/4-event schemas; auto flag any reference with unexpected counts.

- Adaptive outlier bounds: Maintain IQR based caps per state/channel/hour; update weekly.

- Behavioral features: Add rolling velocity, peer group z-scores, and device consistency signals.

- Modeling stack: Use Isolation Forest for prefiltering → Logistic Regression/XGBoost on flagged sets.

- Hotspot playbooks: For Yobe/Oyo, deploy tighter ATM/POS withdrawal thresholds + step-up KYC.

- Explainability & feedback: SHAP for triage; analyst feedback loops to enrich “reported” labels.

- Real-time guardrails: Rule combos (large amount × new device × late hour × cross-state) = hard blocks.